In [2]:
import pandas as pd
import os

In [12]:
# raw data file and path
datafile = '../data/Eraslan-EV3.tsv'

# sanity check if the file exists
if not os.path.isfile(datafile):
    print('Data file not found!')

In [13]:
# reading the data into a dataframe and looking at the first entries
df = pd.read_csv(datafile, sep='\t')
df

,GeneName,EnsemblGeneID,EnsemblTranscriptID,EnsemblProteinID,Adrenal_mRNA,Appendices_mRNA,Brain_mRNA,Colon_mRNA,Duodenum_mRNA,Endometrium_mRNA,...,Rectum_PTR,Salivarygland_PTR,Smallintestine_PTR,Smoothmuscle_PTR,Spleen_PTR,Stomach_PTR,Testis_PTR,Thyroid_PTR,Tonsil_PTR,Urinarybladder_PTR
0,A1BG,ENSG00000121410,ENST00000263100,ENSP00000263100,NA,1.073,NA,NA,NA,NA,...,NA,7.718,NA,NA,7.313,NA,NA,NA,NA,NA
1,A1CF,ENSG00000148584,ENST00000373993,ENSP00000363105,NA,NA,NA,1.971,2.324,NA,...,5.147,NA,5.202,NA,NA,5.8143,NA,NA,NA,NA
2,A2M,ENSG00000175899,ENST00000318602,ENSP00000323929,3.154,3.021,2.824,3.321,3.006,3.344,...,6.081,5.726,5.699,4.997,5.136,6.5349,5.820,6.060,5.675,5.8286
3,A2ML1,ENSG00000166535,ENST00000299698,ENSP00000299698,NA,NA,1.355,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,2.350,NA,5.249,NA
4,A4GALT,ENSG00000128274,ENST00000401850,ENSP00000384794,1.625,1.567,NA,NA,NA,NA,...,4.731,4.508,NA,NA,NA,4.0613,4.832,NA,NA,4.2430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11570,ZXDB,ENSG00000198455,ENST00000374888,ENSP00000364023,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,4.681,NA
11571,ZYG11B,ENSG00000162378,ENST00000294353,ENSP00000294353,1.930,1.589,1.995,1.627,1.531,2.082,...,4.962,4.987,5.076,4.827,4.255,4.0412,5.389,4.250,4.439,4.1460
11572,ZYX,ENSG00000159840,ENST00000322764,ENSP00000324422,2.414,2.978,2.349,2.257,2.572,3.175,...,6.268,5.564,5.708,6.284,6.159,5.8846,5.582,5.598,5.968,5.3358
11573,ZZEF1,ENSG00000074755,ENST00000381638,ENSP00000371051,1.851,1.904,1.866,2.140,2.175,1.689,...,5.540,5.181,5.303,5.038,5.110,5.0834,5.047,5.038,5.130,5.0619


In [14]:
# getting all column names
for col in df.columns:
    print(col)

GeneName
EnsemblGeneID
EnsemblTranscriptID
EnsemblProteinID
Adrenal_mRNA
Appendices_mRNA
Brain_mRNA
Colon_mRNA
Duodenum_mRNA
Endometrium_mRNA
Esophagus_mRNA
Fallopiantube_mRNA
Fat_mRNA
Gallbladder_mRNA
Heart_mRNA
Kidney_mRNA
Liver_mRNA
Lung_mRNA
Lymphnode_mRNA
Ovary_mRNA
Pancreas_mRNA
Placenta_mRNA
Prostate_mRNA
Rectum_mRNA
Salivarygland_mRNA
Smallintestine_mRNA
Smoothmuscle_mRNA
Spleen_mRNA
Stomach_mRNA
Testis_mRNA
Thyroid_mRNA
Tonsil_mRNA
Urinarybladder_mRNA
Adrenal_protein
Appendices_protein
Brain_protein
Colon_protein
Duodenum_protein
Endometrium_protein
Esophagus_protein
Fallopiantube_protein
Fat_protein
Gallbladder_protein
Heart_protein
Kidney_protein
Liver_protein
Lung_protein
Lymphnode_protein
Ovary_protein
Pancreas_protein
Placenta_protein
Prostate_protein
Rectum_protein
Salivarygland_protein
Smallintestine_protein
Smoothmuscle_protein
Spleen_protein
Stomach_protein
Testis_protein
Thyroid_protein
Tonsil_protein
Urinarybladder_protein
Adrenal_PTR
Appendices_PTR
Brain_PTR
Colon_